In [1]:
#Libraries
import re
import numpy as np
import pandas as pd
from matplotlib.pyplot import *
import keras
from keras.models import load_model
from keras.layers import *
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras import backend as K
from keras import layers


import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

In [2]:
def processed_data(data):
    data=data.drop(columns=["Unnamed: 0"])
    data['Utterances'] = data['Utterances'].apply(lambda x: x.lower())
    data['Utterances'] = data['Utterances'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

    y = pd.get_dummies(data['Basic']).values

    return data,y

In [3]:
def DA_label_count(data):
    label_count=data.groupby('Basic')['Basic'].agg('count')
    label_count.to_numpy(dtype=int)
    label_count = np.expand_dims(label_count, axis=0)
    label_count.shape=(5,1)
    labels=np.array(['B', 'D', 'F', 'Q', 'S'])
    labels.shape=(5,1)
    label_count=np.column_stack((labels,label_count))
    df = pd.DataFrame(label_count, columns=['Label', 'Count'])
    convert_dict = {
                    'Label': str,
                    'Count': int
                   }  
    df = df.astype(convert_dict)
    return df

In [4]:
full_data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\full_set.csv')
MAX_PADDING = 100
def x_value(data):
    tokenizer = Tokenizer(num_words = 5000, split=" ")
    tokenizer.fit_on_texts(full_data['Utterances'].values)
    x = tokenizer.texts_to_sequences(data['Utterances'].values)
    x = pad_sequences(x, padding='post', maxlen=MAX_PADDING)
    
    return x

In [5]:
train_data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\train.csv')
train_data,y_train = processed_data(train_data)
x_train = x_value(train_data)

val_data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\val.csv')
val_data,y_val = processed_data(val_data)
x_val = x_value(val_data)

test_data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\test.csv')
test_data,y_test = processed_data(test_data)
x_test = x_value(test_data)

In [6]:
MAX_LENGTH_PER_SENTENCE=100
units = 128
drop = 0.2
encoder_input = keras.Input(shape=(MAX_LENGTH_PER_SENTENCE))

def module(encoder_input):
    x =layers.Embedding(input_dim=5000, 
                        output_dim=256, 
                        input_length=x_train.shape[1])(encoder_input)
                                
    activations = Bidirectional(LSTM(units, 
                                    dropout=0.3, 
                                    recurrent_dropout=0.2,
                                    return_sequences=True))(x)

    attention = Dense(1, activation='tanh')(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(units*2)(attention)
    attention = Permute((2, 1))(attention)

    sent_representation = Multiply()([activations, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), 
                                output_shape=(units*2,))(sent_representation)

    reshape = Reshape((-1, 1, 256))(sent_representation)
    flat = TimeDistributed(Flatten())(reshape)
    dense_1 = Dense(5, activation='relu')(flat)
    dropout_1 = Dropout(drop)(dense_1)


    #conversation level
    biLSTM1 = Bidirectional(LSTM(5, return_sequences='true'))(dropout_1)
    biLSTM2 = Bidirectional(LSTM(5))(biLSTM1)

    dense_2 = Dense(5, activation='relu')(biLSTM2)
    dropout_2 = Dropout(drop)(dense_2)

    # concatenate 2 final layers
    # flatten the output of the CNN + dense + dropout so that it can be concatenated with the output of BiLSTM
    dropout_flat = Flatten()(dropout_1)
    # concatenating the output of CNN + dense + dropout with the output of BiLSTM + dense + dropout
    merged_2 = concatenate([dropout_flat, dropout_2])
    # adding a dense layer
    dense_3 = Dense(units=5, input_shape=(1,))(merged_2)
    # adding softmax for multiclass classification
    output = Activation('softmax')(dense_3)

    return output

In [7]:
model = keras.Model(inputs=[encoder_input], outputs=[module(encoder_input)])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 256)     1280000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 256)     394240      embedding[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 100, 1)       257         bidirectional[0][0]              
______________________________________________________________________________________________

In [8]:
early_stopping = keras.callbacks.EarlyStopping(monitor='accuracy', 
                                               mode='auto', 
                                               patience=1, 
                                               verbose=1)

In [9]:
model.fit(x_train, y_train, 
          epochs=100, 
          batch_size=32, 
          verbose=1,
          callbacks=[early_stopping],
          validation_data=(x_val, y_val))

Epoch 1/100
2346/2346 [==============================] - 1150s 487ms/step - loss: 0.9240 - accuracy: 0.6469 - val_loss: 0.7780 - val_accuracy: 0.6896
Epoch 2/100
2346/2346 [==============================] - 1180s 503ms/step - loss: 0.7605 - accuracy: 0.7111 - val_loss: 0.7408 - val_accuracy: 0.7326
Epoch 3/100
2346/2346 [==============================] - 1273s 543ms/step - loss: 0.7174 - accuracy: 0.7331 - val_loss: 0.7249 - val_accuracy: 0.7317
Epoch 4/100
2346/2346 [==============================] - 1361s 580ms/step - loss: 0.6889 - accuracy: 0.7473 - val_loss: 0.7218 - val_accuracy: 0.7412
Epoch 5/100
2346/2346 [==============================] - 1423s 606ms/step - loss: 0.6934 - accuracy: 0.7461 - val_loss: 0.7110 - val_accuracy: 0.7486
Epoch 00005: early stopping


In [10]:
model.save('SAVED_MODEL.h5')

C:\Users\user1\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [11]:
saved_model = load_model('SAVED_MODEL.h5')

In [12]:
saved_model.evaluate(x_test,y_test)

522/522 [==============================] - 60s 113ms/step - loss: 0.6997 - accuracy: 0.7529


[0.6997259855270386, 0.7529489398002625]

In [13]:
y_pred = saved_model.predict(x_test)

In [14]:
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)


from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[1936    0   22    1  193]
 [ 111 1166  101   50  911]
 [ 197   36  875    1  299]
 [ 259   44    6  420  502]
 [ 864  395   46   88 8178]]


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

ax = sns.heatmap(confusion, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ')

ax.xaxis.set_ticklabels(['B','D','F','Q','S'])
ax.yaxis.set_ticklabels(['B','D','F','Q','S'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [16]:
from sklearn.metrics import f1_score
print("F1 score(macro)",f1_score(y_test, y_pred, average='macro'))
print("F1 score(micro)",f1_score(y_test, y_pred, average='micro'))

F1 score(macro) 0.6601351396329971
F1 score(micro) 0.752948925214059


In [17]:
DA = np.array(['B','D','F','Q','S'])
a = ["will you go"]
tokenizer = Tokenizer(num_words = 5000, split=" ")
full_data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\full_set.csv')
tokenizer.fit_on_texts(full_data['Utterances'].values)
a = tokenizer.texts_to_sequences(a)
a = np.array(a)
a = pad_sequences(a, padding='post', maxlen=MAX_LENGTH_PER_SENTENCE)
prediction = saved_model.predict(np.array(a))
print(DA[prediction.argmax(1)])

['Q']
